In [1]:
import pandas as pd
from collections import Counter
from pypif import pif
from pypif.obj import *

## Optional: Time module and starting time to measure runtime of script.
import time

initialized_time = time.time()

In [2]:
## Defining the start/end file range to convert (for example, dsgdb9nsd_000001.xyz to dsgdb9nsd_010000.xyz)
## This script expects the dsgdb .xyz files to be in a folder named "dsgdb9nsd_files", located in this
## same working directory

## Since we're only trying a representative subset, and we want to show the simpler molecules in the early
## files all the way up to the complex ones near the end, we'll select only every 13th file, which yields
## about 10,000 files to experiment with.

file_path = "/Users/robertmanriquez/Documents/citrine_challenge/dsgdb9nsd_files"
starting_file_no = 1
ending_file_no = 133885

In [3]:
def xyz_to_df(xyz_file):
    
    ### Function for converting an individual .xyz file into a single dataframe row.
    
    ## Grab line 2 from the .xyz, parses to a list.
    scalar_list = [float(i) for i in xyz_file[1].strip('gdb ').strip('\t\n').split('\t')]
    
    ## Grab lines 3 through n_a + 2, only the atom locations and Mulliken charges (x, y, z, e)
    xyz_string =''.join([xyz_file[i] for i in range(2, len(xyz_file)-3)])

    row_dict = {  # Define columns entres for each scalar as a dictionary entry

        "n_atoms": xyz_file[0].strip('\n'),

        "i"      : int(scalar_list[0]),
        "A"      : scalar_list[1],
        "B"      : scalar_list[2],
        "C"      : scalar_list[3],
        "u"      : scalar_list[4],
        "alpha"  : scalar_list[5],
        "HOMO"   : scalar_list[6],
        "LUMO"   : scalar_list[7],
        "HL_gap" : scalar_list[8],
        "<R2>"   : scalar_list[9],
        "zpve"   : scalar_list[10],
        "U_0K"   : scalar_list[11],
        "U_298K" : scalar_list[12],
        "H"      : scalar_list[13],
        "G"      : scalar_list[14],
        "Cv"     : scalar_list[15],
            
        # Storing harmonics, smiles, inchl as lists converted to strings to store in a Pandas df.
        
        "Harmonic_freqs": str([float(i) for i in xyz_file[-3].strip('\n').split('\t')]),

        "SMILES" : str(xyz_file[-2].strip('\t\n').split('\t')),

        "InChI"  : str(xyz_file[-1].strip('\n').split('\t')),

        "atoms_xyz": xyz_string   # Grabbing the whole string for atom locations, will parse into .pif later.
    }
    
    return pd.DataFrame(row_dict, index =[0])

In [4]:
# Optional: Timing the function (does 10k files in ~1.1 min, 100k in 15+ min)
start_time = time.time()

df = xyz_to_df(open(str(file_path) + "/dsgdb9nsd_"\
                    + str(starting_file_no).zfill(6) + ".xyz",'r').readlines())

## Defined starting file, then concatenating the rest to this first one.

for i in range(starting_file_no,ending_file_no + 1):
    
    ## Picking only every 13th file to obtain a "representative" subset
    
    if i%13 == 0:
        loop_file = open(str(file_path) + "/dsgdb9nsd_" \
                        + str(i).zfill(6) + ".xyz", 'r').readlines()

        df = pd.concat([df, xyz_to_df(loop_file)], axis = 0)

## Optional: Print time and number of files inserted into df every 1000th.    
    if i%(1300*5) == 0:
        print(i, round((time.time() - start_time)/60,2), " minutes")
    
df.set_index('i')
df.to_csv('dsgdb_' +str(starting_file_no).zfill(6)+'_to_'+str(ending_file_no).zfill(6)+ '.csv')

6500 0.04  minutes
13000 0.08  minutes
19500 0.12  minutes
26000 0.18  minutes
32500 0.24  minutes
39000 0.3  minutes
45500 0.36  minutes
52000 0.45  minutes
58500 0.53  minutes
65000 0.61  minutes
71500 0.72  minutes
78000 0.82  minutes
84500 0.93  minutes
91000 1.04  minutes
97500 1.15  minutes
104000 1.27  minutes
110500 1.38  minutes
117000 1.49  minutes
123500 1.6  minutes
130000 1.73  minutes


In [5]:
del df  # Delete df after dumping to .csv to save memory.

# Reading it back in from the csv.
df_csv = pd.read_csv('dsgdb_' +str(starting_file_no).zfill(6)+'_to_'+str(ending_file_no).zfill(6)+ '.csv')

In [6]:
## Optional timing function again.  Converted a df of 10k rows in <0.5 min, 15+ min on 100k row df.
start_time = time.time()
chem_list = []

def atomLocParse(atom):
    
### Parses each .xyz atom location line into a list of coordinates in the form of [x, y, z, e]
    
    if atom_dict[atom] > 0:
        return [i[2:].split('\t') for i in df_row['atoms_xyz'].split('\n')[:-1] if i[0] == atom]
    else:
        return None

for idx, df_row in df_csv.iterrows():  # Loop through each row of that dataframe

    chemical = ChemicalSystem()     # Name each individual row as a ChemicalSystem() object to start storing properties

    ## Start: Assigning scalars from line 2 of .xyz file as Property() attributes for ChemicalSystem() ##
    ## Specified from Table 3 of Ramakrishnan1 et al.
    
    A = Property()

    A.name = "Rotaional Constant A"
    A.scalar = df_row['B']
    A.units = "GHz"
    A.data_type = 'COMPUTATIONAL'

    B = Property()

    B.name = "Rotaional Constant B"
    B.scalar = df_row['B']
    B.units = "GHz"
    B.data_type = 'COMPUTATIONAL'

    C = Property()

    C.name = "Rotaional Constant C"
    C.scalar = df_row['C']
    C.units = "GHz"
    C.data_type = 'COMPUTATIONAL'

    u = Property()

    u.name = "Dipole Moment"
    u.scalar = df_row['u']
    u.units = "D"
    u.data_type = 'COMPUTATIONAL'

    alpha = Property()

    alpha.name = "Isotropic polarizability"
    alpha.scalar = df_row['alpha']
    alpha.units = "a3o"
    alpha.data_type = 'COMPUTATIONAL'

    homo = Property()

    homo.name = "Energy of HOMO"
    homo.scalar = df_row['HOMO']
    homo.units = "Ha"
    homo.data_type = 'COMPUTATIONAL'

    lumo = Property()

    lumo.name = "Energy of LUMO"
    lumo.scalar = df_row['LUMO']
    lumo.units = "Ha"
    lumo.data_type = 'COMPUTATIONAL'

    HL_gap = Property()

    HL_gap.name = "Gap (E_LUMO - E_HoMO)"
    HL_gap.scalar = df_row['HL_gap']
    HL_gap.units = "Ha"
    HL_gap.data_type = 'COMPUTATIONAL'


    r2_esa = Property()

    r2_esa.name = "Electronic spatial extent (<R^2>)"
    r2_esa.scalar = df_row['<R2>']
    r2_esa.units = "a2o"
    r2_esa.data_type = 'COMPUTATIONAL'

    zpve = Property()

    zpve.name = "Zero point vibrational energy"
    zpve.scalar = df_row['zpve']
    zpve.units = "Ha"
    zpve.data_type = 'COMPUTATIONAL'


    U_0K = Property()

    U_0K.name = "Internal energy at 0 K"
    U_0K.scalar = df_row['U_0K']
    U_0K.units = "Ha"
    U_0K.data_type = 'COMPUTATIONAL'

    U_298K = Property()

    U_298K.name = "Internal energy at 298.15 K"
    U_298K.scalar = df_row['U_298K']
    U_298K.units = "Ha"
    U_298K.data_type = 'COMPUTATIONAL'

    h = Property()

    h.name = "Enthalpy at 298.15 K"
    h.scalars = df_row['H']
    h.units = "Ha"
    h.data_type = 'COMPUTATIONAL'

    g = Property()

    g.name = "Free energy at 298.15 K"
    g.scalar = df_row['G']
    g.units = "Ha"
    g.data_type = 'COMPUTATIONAL'

    cv = Property()

    cv.name = "Heat capacity at 298.15 K"
    cv.scalar = df_row['Cv']
    cv.units = "cal/mol*K"
    cv.data_type = 'COMPUTATIONAL'

    harmonics = Property()

    harmonics.name = "Harmonic vibrational frequencies"
    harmonics.scalars = [float(i) for i in df_row['Harmonic_freqs'].replace('[','').replace(']','').split(',')]
    harmonics.units = "cm^-1"
    harmonics.data_type = "COMPUTATIONAL"
    
    ## End: Assigning Scalar properties
    
    ## Assigning ID Fields as Id() attributes for ChemicalSystem().

    smiles = Id()

    smiles.name = "SMILES"
    smiles.value = df_row['SMILES'].replace('[','').replace(']','').replace("'","").replace(' ','').split(',')
    
    inchl = Id()
    
    inchl.name = "InChI"
    inchl.value = df_row['InChI']
    
    ### Start: Counting up atoms to assign as Composition() attributes for ChemicalSystem().
    
    # Parsing the .xyz atom locations string to convert into a dictionary.
    
    atom_dict = Counter([atom.split()[0] for atom in df_row['atoms_xyz'].split('\n')[:-1]])
    
    total_atoms = sum(atom_dict.values())
    
    n_C = Composition()

    n_C.element = 'Carbon'
    n_C.idealAtomicPercent = atom_dict['C'] / total_atoms
    n_C.tags = ["n_atoms :", atom_dict['C']]

    n_H = Composition()

    n_H.element = 'Hydrogen'
    n_H.idealAtomicPercent = atom_dict['H'] / total_atoms
    n_H.tags = ["n_atoms :", atom_dict['H']]

    n_O = Composition()

    n_O.element = 'Oxygen'
    n_O.idealAtomicPercent = atom_dict['O'] / total_atoms
    n_O.tags = ["n_atoms :", atom_dict['O']]

    n_N = Composition()

    n_N.element = 'Nitrogen'
    n_N.idealAtomicPercent = atom_dict['N'] / total_atoms
    n_N.tags = ["n_atoms :", atom_dict['N']]
    
    n_F = Composition()

    n_F.element = 'Fluorine'
    n_F.idealAtomicPercent = atom_dict['F'] / total_atoms
    n_F.tags = ["n_atoms :", atom_dict['F']]
    
    total_weight = (
                    12.011 *  n_C.idealAtomicPercent + \
                    1.008  *  n_H.idealAtomicPercent  + \
                    15.999 *  n_O.idealAtomicPercent + \
                    14.007 *  n_N.idealAtomicPercent + \
                    18.998 *  n_F.idealAtomicPercent
                   )
    # Assign ideal weight percents
    
    n_C.ideal_weight_percent = (12.011 *  n_C.idealAtomicPercent) / total_weight
    n_H.ideal_weight_percent = (1.008 *  n_H.idealAtomicPercent) / total_weight
    n_O.ideal_weight_percent = (15.999 *  n_O.idealAtomicPercent) / total_weight
    n_N.ideal_weight_percent = (14.007 *  n_N.idealAtomicPercent) / total_weight
    n_F.ideal_weight_percent = (18.998 *  n_F.idealAtomicPercent) / total_weight
    
    ## End: Assigning atom composition attributes.
    
    ## Start: Atom location coordinate locations and Mulliken charges, assigned as 
    ## Property() attributes to ChemicalSystem()
    
    # Each 
    
    C_xyz = Property()
    
    C_xyz.name = "Carbon atom coordinates and Mulliken partial charges"
    C_xyz.scalars = str(atomLocParse('C'))   # atomLocParse defined above, storing as nested list of coordinates
    C_xyz.units = '(x,y,z in Angstroms; e)'  # for each atom as [ [x,y,z,e], [x,y,z,e], ...]
    C_xyz.data_type = "COMPUTATIONAL"
    
    H_xyz = Property()
    
    H_xyz.name = "Hydrogen atom coordinates and Mulliken partial charges"
    H_xyz.scalars = str(atomLocParse('H'))
    H_xyz.units = '[(x,y,z) in Angstroms, e]'
    H_xyz.data_type = "COMPUTATIONAL"
    
    O_xyz = Property()
    
    O_xyz.name = "Oxygen atom coordinates and Mulliken partial charges"
    O_xyz.scalars = str(atomLocParse('O'))
    O_xyz.units = '[(x,y,z) in Angstroms, e]'
    O_xyz.data_type = "COMPUTATIONAL"
    
    N_xyz = Property()
    
    N_xyz.name = "Nitrogen atom coordinates and Mulliken partial charges"
    N_xyz.scalars = str(atomLocParse('N'))
    N_xyz.units = '[(x,y,z) in Angstroms, e]'
    N_xyz.data_type = "COMPUTATIONAL"
    
    F_xyz = Property()
    
    F_xyz.name = "Fluorine atom coordinates and Mulliken partial charges"
    F_xyz.scalars = str(atomLocParse('F'))
    F_xyz.units = '[(x,y,z) in Angstroms, e]'
    F_xyz.data_type = "COMPUTATIONAL"
    
    ### End: Atom xyz location storing.
    
    ### Building Emperical Chemical Formula
    
    chem_form_string = ""

    elements = zip(['C','H','O','N','F'], \
                   [atom_dict['C'], atom_dict['H'], atom_dict['O'], atom_dict['N'], atom_dict['F']])

    for element, n in elements:
        if n > 0:
            chem_form_string += element + "$_{" + str(n) + "}"
            
    ### Reference & Source Data stored as Reference() and Source() attributes for ChemicalSystems(), respectively.
    
    reference = Reference()

    reference.doi = "10.1038/sdata.2014.22"
    reference.title = "Quantum chemistry structures and properties of 134 kilo molecules"
    reference.journal = 'Sci. Data'
    reference.authors = "Raghunathan Ramakrishnan1, Pavlo O. Dral, Matthias Rupp, O. Anatole von Lilienfeld"
    
    source_data = Source()
    
    source_data.url = "https://figshare.com/articles/Data_for_6095_constitutional_isomers_of_C7H10O2/1057646"
    source_data.producer = "Data for 133885 GDB-9 molecules"
    
    ### Final attribute assignments for ChemicalSystem()
    
    chemical.uid = str(df_row['i'])
    
    chemical.ids = [smiles, inchl]
    
    chemical.chemical_formula = chem_form_string
    
    chemical.properties = [A, B, C, u, alpha, homo, lumo, HL_gap, r2_esa, zpve, \
                           U_0K, U_298K, h, g , cv, harmonics, C_xyz, H_xyz, O_xyz, N_xyz, F_xyz]
    
    chemical.composition = [n_C, n_H, n_O, n_N, n_F]    
    
    chemical.reference = reference
    
    chemical.source = source_data
    
    chem_list.append(chemical)  # Final addition of the converted .xyz file (from df row) into a list entry,
                                # will be exported using pif.dump()
    ## Optional time tracking
    if idx%1000 == 0:
        print(idx, round((time.time() - start_time)/60,2), " minutes")
        
print(round((time.time() - start_time)/60,2), " minutes")

0 0.0  minutes
1000 0.03  minutes
2000 0.05  minutes
3000 0.08  minutes
4000 0.11  minutes
5000 0.13  minutes
6000 0.15  minutes
7000 0.18  minutes
8000 0.2  minutes
9000 0.23  minutes
10000 0.26  minutes
0.26  minutes


In [7]:
### Dumping the list of ChemicalSystem() objects into the desired .pif file, exporting as .pif to root directory

with open('dsgdb_' +str(starting_file_no).zfill(6)+'_to_'+str(ending_file_no).zfill(6)+ '.pif', 'w') as outfile:
    pif.dump(chem_list, outfile)

In [8]:
### Optional:  Final timing of the script runtime.

print(round((time.time() - initialized_time)/60,2), " minutes")
print('finished')

2.36  minutes
finished
